# Bonustask 4: Benders' decomposition

In [113]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `C:\Users\Ferdi`


In [114]:
import Pkg; Pkg.add("GLPK")

   Resolving package versions...
  No Changes to `C:\Users\Ferdi\Project.toml`
  No Changes to `C:\Users\Ferdi\Manifest.toml`


In [175]:
using HiGHS
using JuMP

## Parameter

In [188]:
alphadown = -25;
maxIteration = 15;

## Modelling the main Problem #

In [189]:
masterProblem = Model(HiGHS.Optimizer)
set_silent(masterProblem)
@variable(masterProblem, 0 <= x <= 16)
@variable(masterProblem, α >= alphadown)
@objective(masterProblem, Min, -0.25*x+α)
print(masterProblem)

In [190]:
optimize!(masterProblem)
X=value(x)
α1 = value(α)

println(value(X))
println(value(α1))

16.0
-25.0


## Modelling the sub Problem

In [191]:
function sub(x)
    subProblem = Model(HiGHS.Optimizer)
    set_silent(subProblem)
    @variable(subProblem, y >= 0)
    @objective(subProblem, Min, -y)
    
    @constraint(subProblem, λ, y-x <= 5)
    @constraint(subProblem, γ, y- x/2 <= 15/2)
    @constraint(subProblem, μ, y+ x/2 <= 35/2)
    @constraint(subProblem, β, -y+ x <= 10)
    optimize!(subProblem)
    Zup = -0.25*X-value(y)
    Zlb = -0.25*X + value(α)
    return value(y), shadow_price(λ), shadow_price(γ), shadow_price(μ), shadow_price(β), Zup - Zlb, objective_value(subProblem)
end

sub (generic function with 1 method)

## Solving the Problem

In [192]:
Y, λ, γ, μ, β, gap, objvSub =sub(X)

(9.5, 0.0, 0.0, -1.0, 0.0, 15.5, -9.5)

In [193]:
for k in 1: maxIteration
    println("Iteration: ", k) 
    println("Gap: ", gap)
    println("Objective Value SubProblem: ", objvSub)
    unregister(masterProblem, :duality)
    @constraint(masterProblem, duality[i in 1:maxIteration], α >= -λ*(-5-x) - γ*(-15/2 - 0.5x) - μ*(-35/2 + 0.5x) - β*(-10+x))
    optimize!(masterProblem)
    println("Objective Value MasterProblem: ", objective_value(masterProblem))
    println("x in the nex Iteration: ", value(x))
    println("α in the nex Iteration: ", value(α))
    X=value(x)
    
    
    Y, λ, γ, μ, β, gap, objvSub =sub(X)
    if gap <= 0.000001
        println()
        println()
        println()
        println("The optimal solution has been found! It Took $k iterations.")
        println("Objective Value SubProblem: ", objvSub)
        println("Objective Value MasterProblem: ", objective_value(masterProblem))
        println("Gap: ", gap)
        println("x: ", value(x))
        println("α: ", value(α))
        
        
        
        break
    end
    println()
    
    
end

Iteration: 1
Gap: 15.5
Objective Value SubProblem: -9.5
Objective Value MasterProblem: -17.5
x in the nex Iteration: 0.0
α in the nex Iteration: -17.5

Iteration: 2
Gap: 12.5
Objective Value SubProblem: -5.0
Objective Value MasterProblem: -15.416666666666666
x in the nex Iteration: 8.333333333333334
α in the nex Iteration: -13.333333333333332

Iteration: 3
Gap: 1.6666666666666643
Objective Value SubProblem: -11.666666666666668
Objective Value MasterProblem: -15.0
x in the nex Iteration: 10.0
α in the nex Iteration: -12.5



The optimal solution has been found! It Took 3 iterations
Objective Value SubProblem: -12.5
Objective Value MasterProblem: -15.0
Gap: 0.0
x: 10.0
α: -12.5
